In [1]:
import sys
sys.path.append("../")

import numpy as np
import pandas as pd
import tensorflow as tf
from scipy.stats import pearsonr
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

tf.compat.v1.disable_eager_execution()

from adversarial_debiasing_multi import AdversarialDebiasingMulti

In [2]:
compas = pd.read_csv('data/compas.csv', index_col=False)

In [3]:
def prepare_meta_and_features(df, protected_attribute_name):
    meta = df[[protected_attribute_name, 'label']]
    features = df.drop(columns=[protected_attribute_name, 'label'])
    
    for col in features.columns:
        data = features[col]
        if pd.api.types.is_numeric_dtype(data):
            data -= np.min(data,axis=0)
            data /= (np.max(data,axis=0) - np.min(data,axis=0))
            features[col] = data
        else:
            dummies = pd.get_dummies(data, prefix=col)
            features[col] = dummies
            
    meta['label'] = meta.label.astype(int)
    return meta, features

meta, features = prepare_meta_and_features(compas, 'race')

features_train, features_test, meta_train, meta_test = train_test_split(features, meta, test_size=0.2, random_state=42, stratify=meta.label)

meta_train.reset_index(drop=True, inplace=True)
meta_test.reset_index(drop=True, inplace=True)
features_train.reset_index(drop=True, inplace=True)
features_test.reset_index(drop=True, inplace=True)

/Users/Ari/.virtualenvs/python3/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [4]:
def get_predictions(debias=True, fairness_def='parity', adv_loss_weight=2, prot_attr='race'):
    sess = tf.compat.v1.Session()
    model = AdversarialDebiasingMulti(
        protected_attribute_name=prot_attr,
        num_labels=len(meta_train.label.unique()),
        scope_name='biased_classifier',
        debias=debias,
        adversary_loss_weight=adv_loss_weight,
        fairness_def=fairness_def,
        verbose=False,
        num_epochs=64,
        classifier_num_hidden_units_1=60,
        classifier_num_hidden_units_2=20,
        sess=sess
    )
    model.fit(features_train, meta_train)
    predictions = model.predict(features_test, meta_test)
    sess.close()
    tf.compat.v1.reset_default_graph()
    return predictions

***
# Binary Protected Attribute (Race)

In [5]:
def print_stats(df):
    print('PERFORMANCE:\n')
    print(classification_report(df.label, df.pred_label))
    print('\nBIAS:')
    rw = len(df.loc[(df.race==1) & (df.pred_label==1)]) / len(df.loc[df.race==1])
    print('\nproportion of White people predicted to reoffend: ' + str(rw))
    rn = len(df.loc[(df.race==0) & (df.pred_label==1)]) / len(df.loc[df.race==0])
    print('proportion of Nonwhite people predicted to reoffend: ' + str(rn))
    print('\tRATE GAP = ' + str(rw - rn))
    tprw = len(df.loc[(df.race==1) & (df.pred_label==1) & (df.label==1)]) / len(df.loc[(df.race==1) & (df.label==1)])
    print('\nTPR for White people: ' + str(tprw))
    tprn = len(df.loc[(df.race==0) & (df.pred_label==1) & (df.label==1)]) / len(df.loc[(df.race==0) & (df.label==1)])
    print('TPR for Nonwhite people: ' + str(tprn))
    print('\tTPR GAP = ' + str(tprw - tprn))
    fprw = len(df.loc[(df.race==1) & (df.pred_label==1) & (df.label==0)]) / len(df.loc[(df.race==1) & (df.label==0)])
    print('\nFPR for White people: ' + str(fprw))
    fprn = len(df.loc[(df.race==0) & (df.pred_label==1) & (df.label==0)]) / len(df.loc[(df.race==0) & (df.label==0)])
    print('FPR for Nonwhite people: ' + str(fprn))
    print('\tFPR GAP = ' + str(fprw - fprn))

## Baseline

In [6]:
predictions = get_predictions(debias=False)
print_stats(predictions)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
PERFORMANCE:

              precision    recall  f1-score   support

           0       0.68      0.71      0.70       672
           1       0.64      0.60      0.62       562

    accuracy                           0.66      1234
   macro avg       0.66      0.66      0.66      1234
weighted avg       0.66      0.66      0.66      1234


BIAS:

proportion of White people predicted to reoffend: 0.31386861313868614
proportion of Nonwhite people predicted to reoffend: 0.4835965978128797
	RATE GAP = -0.1697279846741936

TPR for White people: 0.4472049689440994
TPR for Nonwhite people: 0.655860349127182
	TPR GAP = -0.20865538018308266

FPR for White people: 0.228
FPR for Nonwhite people: 0.31990521327014215
	FPR GAP = -0.09190521327014214


## Parity Fairness

In [7]:
predictions = get_predictions(fairness_def='parity', adv_loss_weight=15)
print_stats(predictions)

PERFORMANCE:

              precision    recall  f1-score   support

           0       0.69      0.61      0.65       672
           1       0.59      0.67      0.63       562

    accuracy                           0.64      1234
   macro avg       0.64      0.64      0.64      1234
weighted avg       0.65      0.64      0.64      1234


BIAS:

proportion of White people predicted to reoffend: 0.44038929440389296
proportion of Nonwhite people predicted to reoffend: 0.551640340218712
	RATE GAP = -0.11125104581481904

TPR for White people: 0.5527950310559007
TPR for Nonwhite people: 0.71571072319202
	TPR GAP = -0.16291569213611934

FPR for White people: 0.368
FPR for Nonwhite people: 0.3957345971563981
	FPR GAP = -0.02773459715639809


## Equal Odds Fairness

In [8]:
predictions = get_predictions(fairness_def='equal_odds', adv_loss_weight=50)
print_stats(predictions)

PERFORMANCE:

              precision    recall  f1-score   support

           0       0.66      0.53      0.59       672
           1       0.54      0.67      0.60       562

    accuracy                           0.59      1234
   macro avg       0.60      0.60      0.59      1234
weighted avg       0.61      0.59      0.59      1234


BIAS:

proportion of White people predicted to reoffend: 0.48418491484184917
proportion of Nonwhite people predicted to reoffend: 0.5990279465370595
	RATE GAP = -0.11484303169521032

TPR for White people: 0.5900621118012422
TPR for Nonwhite people: 0.7007481296758105
	TPR GAP = -0.11068601787456822

FPR for White people: 0.416
FPR for Nonwhite people: 0.5023696682464455
	FPR GAP = -0.08636966824644549


***
# Continuous Protected Attribute (Age)

In [9]:
meta, features = prepare_meta_and_features(compas, 'age')

features_train, features_test, meta_train, meta_test = train_test_split(features, meta, test_size=0.2, random_state=42, stratify=meta.label)

meta_train.reset_index(drop=True, inplace=True)
meta_test.reset_index(drop=True, inplace=True)
features_train.reset_index(drop=True, inplace=True)
features_test.reset_index(drop=True, inplace=True)

/Users/Ari/.virtualenvs/python3/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [10]:
def print_stats(df):
    print('PERFORMANCE:\n')
    print(classification_report(df.label, df.pred_label))
    print('\nBIAS:')
    corr = pearsonr(df.age, df.pred_label)[0]
    corr_1 = pearsonr(df.loc[df.label==1].age, df.loc[df.label==1].pred_label)[0]
    corr_0 = pearsonr(df.loc[df.label==0].age, df.loc[df.label==0].pred_label)[0]
    print('\nCorrelation between age and predicted label: ' + str(corr))
    print('\nCorrelation between age and predicted label, conditional on true label=1: ' + str(corr_1))
    print('\nCorrelation between age and predicted label, conditional on true label=0: ' + str(corr_0))

## Baseline

In [11]:
predictions = get_predictions(debias=False, prot_attr='age')
print_stats(predictions)

PERFORMANCE:

              precision    recall  f1-score   support

           0       0.68      0.71      0.70       672
           1       0.64      0.61      0.62       562

    accuracy                           0.67      1234
   macro avg       0.66      0.66      0.66      1234
weighted avg       0.66      0.67      0.66      1234


BIAS:

Correlation between age and predicted label: -0.2342970206582865

Correlation between age and predicted label, conditional on true label=1: -0.12799579391774052

Correlation between age and predicted label, conditional on true label=0: -0.23857925209720993


## Parity Fairness

In [12]:
predictions = get_predictions(prot_attr='age', fairness_def='parity', adv_loss_weight=0.00001)
print_stats(predictions)

PERFORMANCE:

              precision    recall  f1-score   support

           0       0.71      0.23      0.34       672
           1       0.49      0.89      0.63       562

    accuracy                           0.53      1234
   macro avg       0.60      0.56      0.49      1234
weighted avg       0.61      0.53      0.47      1234


BIAS:

Correlation between age and predicted label: 0.024079090559782818

Correlation between age and predicted label, conditional on true label=1: 0.04695278610291956

Correlation between age and predicted label, conditional on true label=0: 0.05651431668737571


## Equal Odds Fairness

In [13]:
predictions = get_predictions(prot_attr='age', fairness_def='equal_odds', adv_loss_weight=0.001)
print_stats(predictions)

PERFORMANCE:

              precision    recall  f1-score   support

           0       0.63      0.86      0.73       672
           1       0.70      0.40      0.51       562

    accuracy                           0.65      1234
   macro avg       0.67      0.63      0.62      1234
weighted avg       0.66      0.65      0.63      1234


BIAS:

Correlation between age and predicted label: -0.1621362925253865

Correlation between age and predicted label, conditional on true label=1: -0.11367470319343426

Correlation between age and predicted label, conditional on true label=0: -0.12385043989146062
